# 네이버 뉴스 핵심키워드 추출
* 입력 : 네이버 뉴스 url, 핵심 키워드 갯수
* 방법 : matrix 혹은 그래프 활용 textrank 구현 이용한 문서 요약
* 출력 : 핵심키워드 리스트(갯수 적용), 원문

In [90]:
import requests
from bs4 import BeautifulSoup as bs
import re
from konlpy.tag import Komoran
import pandas as pd
import numpy as np
import math
from collections import Counter
from collections import defaultdict

In [184]:
url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=277&aid=0004511484'
resp = requests.get(url)
soup = bs(resp.content)
Text = soup.select('div._article_body_contents')[0].text\
.replace('// flash 오류를 우회하기 위한 함수 추가','')\
.replace('function _flash_removeCallback() {}','').strip()
body = re.sub(r'\[\w{0,10} ?[가-힣]{2,5} ? 기자|\]|\n|\t|\w{0,5} ?기자 ?[a-zA-Z]*@[a-zA-Z]*.*','', Text)


In [183]:

for i in range(len(komoTokens)):
    if komoTokens[i][1]=='NNG' and komoTokens[i+1][1] == 'XSN':
        komoTokens[i] = re.sub("\', \'NNG\'\) ?\(\'", "", str(komoTokens[i])+str(komoTokens[i+1]))
        komoTokens[i] = eval(re.sub("XSN", "NNG", komoTokens[i]))
stopPos = re.sub("\(.{1,15}\'(E\w{1,2}|J\w{1,2}|XS\w|SE|SF|SO|SP|SS|SW|VCP)\'\),? ?","",str(komoTokens))
stopwords = re.sub("\(\'(등|들|있)\', \'\w{1,3}\'\),? ?","", stopPos)
onlyText = re.sub("\(|,? ?\'\w{1,3}\'\)","",stopwords)
prodText = eval(onlyText)
voca = list(set(prodText))
voca_len = len(voca)

Counter(prodText).most_common()

[('글꼴', 22),
 ('부리', 14),
 ('화면용용', 7),
 ('사용자', 6),
 ('개발', 6),
 ('마루', 6),
 ('한글', 6),
 ('네이버', 5),
 ('새롭', 5),
 ('설계', 5),
 ('디지털', 4),
 ('환경', 4),
 ('함께', 4),
 ('이', 4),
 ('화면', 4),
 ('다양', 4),
 ('프로젝트', 3),
 ('예정', 3),
 ('맞추', 2),
 ('무료', 2),
 ('한', 2),
 ('잇', 2),
 ('의미', 2),
 ('나가', 2),
 ('게', 2),
 ('크', 2),
 ('안상수', 2),
 ('디자이너', 2),
 ('일반', 2),
 ('따르', 2),
 ('체', 2),
 ('민', 2),
 ('표현', 2),
 ('담기', 2),
 ('미감', 2),
 ('오늘날', 2),
 ('해상도', 2),
 ('오', 2),
 ('기술', 2),
 ('완성', 2),
 ('높', 2),
 ('매체', 2),
 ('필요', 2),
 ('가독성', 2),
 ('유용성', 2),
 ('개념', 2),
 ('수', 2),
 ('형태', 2),
 ('계획', 2),
 ('사용자들들', 2),
 ('분석', 2),
 ('진행', 2),
 ('하', 2),
 ('방식', 2),
 ('기술력', 1),
 ('더하', 1),
 ('2021년', 1),
 ('공개', 1),
 ('원형', 1),
 ('1', 1),
 ('일', 1),
 ('밝히', 1),
 ('초기', 1),
 ('단계', 1),
 ('방향', 1),
 ('고민', 1),
 ('뼈대', 1),
 ('타이포그래피', 1),
 ('조선', 1),
 ('시대', 1),
 ('붓', 1),
 ('다듬', 1),
 ('지', 1),
 ('궁', 1),
 ('중', 1),
 ('해서', 1),
 ('인쇄용용', 1),
 ('활자', 1),
 ('맞', 1),
 ('정리', 1),
 ('글자', 1),
 ('줄기', 1),
 ('없'

In [188]:
komoran= Komoran()
komoTokens = komoran.pos(body)

In [248]:
def getNewsBody(url):
    resp = requests.get(url)
    soup = bs(resp.content)
    Text = soup.select('div._article_body_contents')[0].text\
    .replace('// flash 오류를 우회하기 위한 함수 추가','')\
    .replace('function _flash_removeCallback() {}','').strip()
    body = re.sub(r'\[\w{0,10} ?[가-힣]{2,5} ? 기자|\]|\n|\t|\w{0,5} ?기자 ?[a-zA-Z]*@[a-zA-Z]*.*','', Text)
    return body

    
def keywords(body):
    komoran= Komoran()
    komoTokens = komoran.pos(body)
    
    for i in range(len(komoTokens)):
        if komoTokens[i][1]=='NNG' and komoTokens[i+1][1] == 'XSN':
            komoTokens[i] = re.sub("\', \'NNG\'\) ?\(\'", "", str(komoTokens[i])+str(komoTokens[i+1]))
            komoTokens[i] = eval(re.sub("XSN", "NNG", komoTokens[i]))
    stopPos = re.sub("\(.{1,15}\'(E\w{1,2}|J\w{1,2}|XS\w|SE|SF|SO|SP|SS|SW|VCP)\'\),? ?","",str(komoTokens))
    stopwords = re.sub("\(\'(등|들|있)\', \'\w{1,3}\'\),? ?","", stopPos)
    onlyText = re.sub("\(|,? ?\'\w{1,3}\'\)","",stopwords)
    prodText = eval(onlyText)
    return Counter(prodText).most_common()

def keywordsRank(komoTokens, count = 10, win = 3):
    komoran= Komoran()
    komoTokens = komoran.pos(body)
    
    # 불용어 처리
    for i in range(len(komoTokens)):
        if komoTokens[i][1]=='NNG' and komoTokens[i+1][1] == 'XSN':
            komoTokens[i] = re.sub("\', \'NNG\'\) ?\(\'", "", str(komoTokens[i])+str(komoTokens[i+1]))
            komoTokens[i] = eval(re.sub("XSN", "NNG", komoTokens[i]))
    stopPos = re.sub("\(.{1,15}\'(E\w{1,2}|J\w{1,2}|XS\w|SE|SF|SO|SP|SS|SW|VCP)\'\),? ?","",str(komoTokens))
    stopwords = re.sub("\(\'(등|들|있)\', \'\w{1,3}\'\),? ?","", stopPos)
    onlyText = re.sub("\(|,? ?\'\w{1,3}\'\)","",stopwords)
    prodText = eval(onlyText)
    voca = list(set(prodText))
    voca_len = len(voca)
    
    # edge 연결
    edge = pd.DataFrame(np.zeros((voca_len, voca_len), dtype = np.float32), columns = voca, index = voca)
        
    for start in range(voca_len-win+1):
        scene = prodText[start:start+win]
        for w1 in scene:
            for w2 in scene:
                if w1 == w2:
                    continue
                else:
                    edge[w1][w2]=1
                    edge[w2][w1]=1
    
    # ranking score 계산
    summ = edge.sum(axis =1 )
    summ = summ[summ !=0] # 연결 없는 노드 제거
    edge = edge[list(summ.index)] 

    cnt = 0
    Max = 50
    d = 0.85
    threshold = 0.0001
    score = np.ones((len(summ)), dtype = np.float32)

    while cnt != Max:
        cnt+=1
        pre_score = np.copy(score)
        weight = []
        for i in range(len(summ)):
            if summ[i] != 0:
                ser = pre_score[i]* edge.iloc[i]/summ[i]
                weight.append(ser)

        weight = pd.DataFrame(weight)
        score = (1-d) + d*weight.sum()
        if np.fabs(pre_score-score).sum() <= threshold:
            break

    keyList = list(score.sort_values(ascending = False)[:count].index)
    return keyList

In [249]:
body = getNewsBody(url)
body

'디지털 환경 맞춰 기술력 더해 사용자와 함께 개발…2021년 무료 공개 네이버는 디지털 환경에 맞춰 한글꼴의 원형을 잇는 화면용 \'마루 부리 글꼴\'을 개발한다고 1일 밝혔다. 프로젝트 초기 단계부터 한글 사용자와 함께 한글꼴의 의미와 방향을 고민하고 새로운 화면용 글꼴을 설계해 나간다는 게 큰 뼈대다. 안상수 한글 타이포그래피 디자이너와 일반 한글 사용자가 함께 새로운 화면용 \'마루 부리 글꼴\'을 설계할 예정이다.네이버에 따르면 부리 글꼴은 조선시대 붓으로 다듬어진 궁체 중 해서체를 인쇄용 활자에 맞게 정리한 글꼴로, 글자 줄기에 부리가 없는 민부리 글꼴과 차이가 있다. 부리 글꼴은 서예에 기본을 두고 있어 손글씨와 같이 미세한 필압 표현과 높낮이가 있는 둥근 획, 감정이 담긴 섬세한 미감을 표현한다. 이에 인간적이고 친숙하며 따뜻한 감성이 담겨 있어 오늘날 신문, 잡지, 동화책 등의 인쇄 매체에 주로 쓰이고 있다.1990년대 화면용 한글 글꼴은 해상도의 한계 등으로 인해 저해상도 화면에서도 일그러짐이 적은 민부리 글꼴을 중심으로 개발돼 왔다. 하지만 디지털 화면 출력 기술이 발전함에 따라 완성도 높은 부리 글꼴 개발에 대한 요구가 커지고 있다. 이에 다양한 디지털 매체 환경에서는 완성도 높은 화면용 부리 글꼴의 개발이 반드시 필요하다는 게 네이버의 설명이다.\'마루 부리 글꼴\' 디자인은 크게 확장성, 가독성, 유용성 3가지 기준으로 설계될 예정이다. 그동안 전통적 의미에서 해석되던 부리의 개념을 확장해 새로운 미감과 안정감을 담는다. 또한 스마트폰에서 긴 글을 잘 읽을 수 있도록 화면에 알맞고 눈이 편안한 글꼴 형태와 구조로 가독성을 높일 방침이다. 이를 바탕으로 다양한 포맷을 지원해 한글 사용자의 유용성을 향상시킬 젊은 \'마루 부리 글꼴\'을 개발한다는 계획이다. 마루 부리 글꼴은 2021년 일반 한글 사용자들에게 무료로 배포된다.네이버는 이를 위해 지난해부터 동아시아 문화권의 글꼴 현황 분석과 화면용 글꼴 형태 및 공간 분석을 진행해왔다. 8

In [250]:
keywords(body)

[('글꼴', 22),
 ('부리', 14),
 ('화면용', 7),
 ('사용자', 6),
 ('개발', 6),
 ('마루', 6),
 ('한글', 6),
 ('네이버', 5),
 ('새롭', 5),
 ('설계', 5),
 ('디지털', 4),
 ('환경', 4),
 ('함께', 4),
 ('이', 4),
 ('화면', 4),
 ('다양', 4),
 ('프로젝트', 3),
 ('예정', 3),
 ('맞추', 2),
 ('무료', 2),
 ('한', 2),
 ('잇', 2),
 ('의미', 2),
 ('나가', 2),
 ('게', 2),
 ('크', 2),
 ('안상수', 2),
 ('디자이너', 2),
 ('일반', 2),
 ('따르', 2),
 ('체', 2),
 ('민', 2),
 ('표현', 2),
 ('담기', 2),
 ('미감', 2),
 ('오늘날', 2),
 ('해상도', 2),
 ('오', 2),
 ('기술', 2),
 ('완성', 2),
 ('높', 2),
 ('매체', 2),
 ('필요', 2),
 ('가독성', 2),
 ('유용성', 2),
 ('개념', 2),
 ('수', 2),
 ('형태', 2),
 ('계획', 2),
 ('사용자들', 2),
 ('분석', 2),
 ('진행', 2),
 ('하', 2),
 ('방식', 2),
 ('기술력', 1),
 ('더하', 1),
 ('2021년', 1),
 ('공개', 1),
 ('원형', 1),
 ('1', 1),
 ('일', 1),
 ('밝히', 1),
 ('초기', 1),
 ('단계', 1),
 ('방향', 1),
 ('고민', 1),
 ('뼈대', 1),
 ('타이포그래피', 1),
 ('조선', 1),
 ('시대', 1),
 ('붓', 1),
 ('다듬', 1),
 ('지', 1),
 ('궁', 1),
 ('중', 1),
 ('해서', 1),
 ('인쇄용', 1),
 ('활자', 1),
 ('맞', 1),
 ('정리', 1),
 ('글자', 1),
 ('줄기', 1),
 ('없', 1

In [251]:
win2 = keywordsRank(body,15,2)
win3 = keywordsRank(body,15,3)
win4 = keywordsRank(body,15,4)
win5 = keywordsRank(body,15,5)

In [256]:
win1 = ['글꼴', '부리','화면용','사용자','개발','마루','한글','네이버','새롭','설계','디지털','환경','함께','이','화면']

In [257]:
print(win1, win2, win3, win4, win5)

['글꼴', '부리', '화면용', '사용자', '개발', '마루', '한글', '네이버', '새롭', '설계', '디지털', '환경', '함께', '이', '화면'] ['글꼴', '부리', '화면용', '개발', '새롭', '체', '네이버', '마루', '담기', '표현', '디지털', '개념', '화면', '사용자', '확장'] ['글꼴', '부리', '화면용', '개발', '새롭', '디지털', '네이버', '마루', '표현', '완성', '담기', '함께', '설계', '사용자', '게'] ['글꼴', '부리', '개발', '화면용', '새롭', '표현', '설계', '네이버', '마루', '담기', '사용자', '디지털', '따르', '의미', '완성'] ['글꼴', '부리', '개발', '화면용', '표현', '디지털', '담기', '마루', '설계', '새롭', '의미', '함께', '환경', '네이버', '따르']


In [413]:

def get_ngram_counter(docs, min_count=10, n_range=(1,3)):
    def to_ngrams(words, n):
        ngrams = []
        for b in range(0, len(words) - n + 1):
            ngrams.append(tuple(words[b:b+n]))
        return ngrams

    n_begin, n_end = n_range
    ngram_counter = defaultdict(int)

    words = komoran.pos(docs)
    for n in range(n_begin, n_end + 1):
        for ngram in to_ngrams(words, n):
            ngram_counter[ngram] += 1

    ngram_counter = {
        ngram:count for ngram, count in ngram_counter.items()
        if count >= min_count
    }

    return ngram_counter

ngram_counter = get_ngram_counter(body,3,(1,5))
ngram_counter